In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from datetime import datetime
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

In [2]:
def average(df):
    sum = 0
    count = 0
    for x in df['f1']:
        if x!=-1:
            sum+=x
            count+=1
    avg = sum/count
    return avg


In [3]:
def results(model, X_train, X_valid,y_train, y_valid):
    soft_probs = model.predict_proba(X_valid)
    valid_auc = roc_auc_score(y_valid, soft_probs[:,1])
    print(valid_auc)
    train_preds = model.predict(X_train)
    acc = accuracy_score(y_train, train_preds)
    print(acc)
    preds = model.predict(X_valid)
    acc = accuracy_score(y_valid, preds)
    print(acc)
    print(classification_report(y_valid,preds))

In [4]:
df = pd.read_csv("train_final.csv")
df.describe()
f1_avg = average(df)
df['f1'].replace(-1,f1_avg, inplace = True)
df.describe()

,Id,Y,f1,f2,f3,f4,f5,f6,f7,f8,...,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24
count,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,...,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,1.638300e+04,16383.000000
mean,8192.000000,0.942135,43031.415720,1.044375,11.770938,118323.581456,1.044436,0.050052,117089.674113,169730.178600,...,25894.316914,119045.099005,184622.040835,1.047305,125959.667765,1.044558,1.045718,1.041934,3.271890e+04,1.043948
std,4729.509065,0.233495,33596.053696,0.264806,353.187115,4518.059755,0.265601,0.293892,10261.292970,69396.677853,...,36086.993946,18321.987129,100590.811845,0.306239,31091.344158,0.262576,0.266874,0.246597,3.184929e+06,0.259640
min,1.000000,0.000000,37.000000,1.000000,1.770000,23779.000000,1.000000,0.000000,4292.000000,4673.000000,...,25.000000,4674.000000,3130.000000,1.000000,117879.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000
25%,4096.500000,1.000000,20331.000000,1.000000,1.770000,118096.000000,1.000000,0.000000,117961.000000,117906.000000,...,4554.000000,118395.000000,118398.000000,1.000000,118274.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000
50%,8192.000000,1.000000,35530.000000,1.000000,1.770000,118300.000000,1.000000,0.000000,117961.000000,128130.000000,...,13234.000000,118929.000000,119095.000000,1.000000,118568.000000,1.000000,1.000000,1.000000,2.000000e+00,1.000000
75%,12287.500000,1.000000,74240.500000,1.000000,3.540000,118386.000000,1.000000,0.000000,117961.000000,234498.500000,...,38902.000000,120539.000000,290919.000000,1.000000,120006.000000,1.000000,1.000000,1.000000,9.000000e+00,1.000000
max,16383.000000,1.000000,312152.000000,7.000000,43910.160000,286791.000000,9.000000,10.000000,311178.000000,311867.000000,...,311696.000000,286792.000000,308574.000000,18.000000,311867.000000,8.000000,8.000000,7.000000,4.042886e+08,8.000000


In [5]:
y = df.loc[:,'Y']
X = df.loc[:,'f1':'f24']
X_train, X_valid, y_train, y_valid = train_test_split(X,y,train_size = 0.75, test_size = 0.25,random_state = 42, shuffle = True)

(colsample_bytree=0.3,subsample = 0.7,max_depth=8,
                            n_estimators=1550, learning_rate =0.011,
                            colsample_bylevel=0.5,n_jobs=-1,base_score = 0.55,
                            random_state=42,)
                     (colsample_bytree=0.4,subsample = 1,max_depth=7,
                            n_estimators=1350, learning_rate =0.012,
                            colsample_bylevel=0.6,n_jobs=-1,base_score = 0.55,
                            random_state=42)
                            

In [88]:
model_trial = XGBClassifier(colsample_bytree=0.4,subsample = 1,max_depth=7,
                            n_estimators=1350, learning_rate =0.012,
                            colsample_bylevel=0.6,n_jobs=-1,base_score = 0.55,
                            random_state=42)
                            
model_trial.fit(X_train, y_train)
results(model_trial,X_train, X_valid, y_train, y_valid)

0.868150226903778
0.9725726377472125
0.9609375
              precision    recall  f1-score   support

           0       0.93      0.31      0.46       224
           1       0.96      1.00      0.98      3872

    accuracy                           0.96      4096
   macro avg       0.95      0.65      0.72      4096
weighted avg       0.96      0.96      0.95      4096



Although the auc score is slighly less, the precision and recal is slightly better than the current best model. Lets make a submission to see if its worth relying on precision/recall

In [50]:
model_trial.fit(X,y)

XGBClassifier(base_score=0.55, booster='gbtree', colsample_bylevel=0.6,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.012, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1350, n_jobs=-1, num_parallel_tree=1,
              random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [51]:
testdf = pd.read_csv("test_final.csv")
testdf['f1'].replace(-1,f1_avg, inplace = True)
X_test = testdf.loc[:,'f1':'f24']
test_preds = model_trial.predict_proba(X_test)
sub6 = {"Id":testdf['Id'],"Y":test_preds[:,1]}
sub6 = pd.DataFrame(data=sub6)
sub6.to_csv("submissions/sub6.csv", index=False)

 Slight improvement noticed in LB. Score now is 0.90633 Stack with lightgbm now

In [53]:
from lightgbm import LGBMClassifier


In [84]:
model_lgb=LGBMClassifier(colsample_bytree=0.9,subsample =0.5,max_depth=-1,
                            n_estimators=1850, learning_rate =0.01)
model_lgb.fit(X_train,y_train)
results(model_lgb,X_train, X_valid, y_train, y_valid)

[LightGBM] [Warning] bagging_fraction is set=0.331, subsample=0.5 will be ignored. Current value: bagging_fraction=0.331
0.8509583456316412
0.9598762920159518
0.9619140625
              precision    recall  f1-score   support

           0       1.00      0.30      0.47       224
           1       0.96      1.00      0.98      3872

    accuracy                           0.96      4096
   macro avg       0.98      0.65      0.72      4096
weighted avg       0.96      0.96      0.95      4096

